# Bulk extract mean fluorescence and process labview data

In [1]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
import numpy as np
import labrotation.file_handling as fh
import os
import pandas as pd
import labrotation.two_photon_session as tps

In [3]:
export_results_folder = fh.open_dir("Choose folder to save results")

In [4]:
data_docu_folder = fh.open_dir("Open Data Documentation folder")

In [5]:
matlab_2p_folder = fh.open_dir("Open matlab-2p folder") 

In [6]:
docu_grp_files_list = []
docu_seg_files_list = []
dframes_grouping = []
dframes_segmentation = []

session_uuid = None
for root, dirs, files in os.walk(data_docu_folder):
    for name in files:
        if "grouping" in name:
            if "~" in name: # "~" on windows is used for temporary files that are opened in excel
                docu_grp_files_list = []
                docu_seg_files_list = []
                dframes_grouping = []
                dframes_segmentation = []
                raise Exception(f"Please close all excel files and try again. Found temporary file in:\n{os.path.join(root, name)}")
            else:
                fpath = os.path.join(root, name)
                df = pd.read_excel(fpath)
                docu_grp_files_list.append(fpath)
                dframes_grouping.append(df)
        elif "segmentation" in name:
            if "~" in name: # "~" on windows is used for temporary files that are opened in excel
                docu_grp_files_list = []
                docu_seg_files_list = []
                dframes_grouping = []
                dframes_segmentation = []
                raise Exception(f"Please close all excel files and try again. Found temporary file in:\n{os.path.join(root, name)}")
            else:
                fpath = os.path.join(root, name)
                df = pd.read_excel(fpath)
                docu_seg_files_list.append(fpath)
                dframes_segmentation.append(df)

In [7]:
df_grouping = pd.concat(dframes_grouping)
df_segmentation = pd.concat(dframes_segmentation)

In [8]:
df_grouping.head()

,folder,nd2,labview,lfp,face_cam_last,nikon_meta,experiment_type,day,uuid
0,Y:\AG-Wenzel\Group\tmev\T301\T301_base_d1,T301_base_d1.180820.1614.nd2,T301_base_d1.180820.1614.txt,NaN,T301_base_d1.180820.1614_2.avi,T301_base_d1.180820.1614_nik.txt,tmev_bl,bl1,3183503d487f428fbbfe3b8bb4fb2292
1,Y:\AG-Wenzel\Group\tmev\T301\T301_base_d1,T301_base_d1.180820.1636.nd2,T301_base_d1.180820.1636.txt,NaN,T301_base_d1.180820.1636_2.avi,T301_base_d1.180820.1636_nik.txt,tmev_bl,bl1,3bc014442e4f42cbaf7f24ec71508b6b
2,Y:\AG-Wenzel\Group\tmev\T301\T301_base_d1,T301_dual_color_zstack_01.nd2,NaN,NaN,NaN,NaN,tmev_stack_dual_2,bl1,1b22fe5249484fe59c5cf24591ba5fd0
3,Y:\AG-Wenzel\Group\tmev\T301\T301_base_d1,T301_dual_color_zstack_02.nd2,NaN,NaN,NaN,NaN,tmev_stack_dual_8,bl1,89eac973f25b4d4a9db404cae5ef3f10
4,Y:\AG-Wenzel\Group\tmev\T301\T301_base_d1,T301_dual_color_tseries_2min.nd2,NaN,NaN,NaN,NaN,fov_dual,bl1,c5f402a018cc4359b5cdb80586102647


In [12]:
uuids_sz = ["171693d0988c458a96c8198c7b8cfc28",
"171693d0988c458a96c8198c7b8cfc28",
"a6099849121f44ccbec237037971ab57",
"a6099849121f44ccbec237037971ab57",
"a6099849121f44ccbec237037971ab57",
"e40f26d410ab452e8f8d59e5394ae0fe",
"e40f26d410ab452e8f8d59e5394ae0fe",
"e40f26d410ab452e8f8d59e5394ae0fe",
"ae564f8c867f4f35aa971b6562c33a7c",
"ae564f8c867f4f35aa971b6562c33a7c",
"ae564f8c867f4f35aa971b6562c33a7c",
"65bff16a4cf04930a5cb14f489a8f99b",
"30dc55d1a5dc4b0286d132e72f208ca6",
"30dc55d1a5dc4b0286d132e72f208ca6",
"30dc55d1a5dc4b0286d132e72f208ca6",
"d7a5ac8e2bc74382b3db503a6a5a07a5",
"d7a5ac8e2bc74382b3db503a6a5a07a5",
"d7a5ac8e2bc74382b3db503a6a5a07a5",
"5ea6fd9c4cb542dbbc1f65305725cede",
"5ea6fd9c4cb542dbbc1f65305725cede",
"5ea6fd9c4cb542dbbc1f65305725cede",
"06ebcf354f5c41519669f187e16de364",
"06ebcf354f5c41519669f187e16de364",
"06ebcf354f5c41519669f187e16de364",
"73a27053f4bf4ae1b4ad96064b6dabc0",
"73a27053f4bf4ae1b4ad96064b6dabc0",
"73a27053f4bf4ae1b4ad96064b6dabc0",
"2aa75aa234a749668eb896e7e00aa87a",
"2aa75aa234a749668eb896e7e00aa87a",
"79fb974821f34e3abdcf5ca650e1c0f4",
"79fb974821f34e3abdcf5ca650e1c0f4",
"79fb974821f34e3abdcf5ca650e1c0f4",
"4fe45b25dc854453880cd868fe77e9d4",
"4fe45b25dc854453880cd868fe77e9d4",
"4fe45b25dc854453880cd868fe77e9d4",
"4fe45b25dc854453880cd868fe77e9d4",
"4fe45b25dc854453880cd868fe77e9d4",
"4fe45b25dc854453880cd868fe77e9d4",
"ba6bb6298c474451be3237a6afe7ffe9",
"ba6bb6298c474451be3237a6afe7ffe9",
"ba6bb6298c474451be3237a6afe7ffe9",
"8dd54649e47046239ebafc56eeb8b5b2",
"44ca941252064dcabb0fe3d24a8dab49",
"3dd896d33a0f42c698228fbe254ebd60",
"a9694ce2973349cb9cb6b51f77c46b49",
"b8f31023d2c042c2a7f95b54d9807cb7",
"757c430daa2349e198ddefa7a0277769",
"92062a977958443e83011619b34eabb8"]

for iterrow in df_grouping.iterrows():  # tuple of (index, row)
    i_row, row = iterrow
    uuid = row["uuid"]
    if uuid not in uuids_sz:
        continue
    export_h5fpath = os.path.join(export_results_folder, str(uuid) + "_FluLoco.h5")
    
    
    if os.path.exists(export_h5fpath):
        print(f"Results file for {uuid} already exists; skipping session.")
        continue
    if type(row["nd2"]) is not float:  # NaN entries are of type float
        nd2_fpath = os.path.join(row["folder"], row["nd2"])
        assert os.path.exists(nd2_fpath)
    else:
        nd2_fpath = None
    if type(row["labview"]) is not float:  # NaN entries are of type float
        labview_fpath = os.path.join(row["folder"], row["labview"])
        labview_tstamps_fpath = os.path.join(row["folder"], os.path.splitext(row["labview"])[0] + "time.txt")  # infer labview timestamps file name
        assert os.path.exists(labview_fpath)
    else:
        labview_fpath = None
        labview_tstamps_fpath = None
    if type(row["nikon_meta"]) is not float:  # NaN entries are of type float
        nikmeta_fpath = os.path.join(row["folder"], row["nikon_meta"])
        assert os.path.exists(nikmeta_fpath)
    else:
        nikmeta_fpath = None
    if type(row["lfp"]) is not float:  # NaN entries are of type float
        lfp_fpath = os.path.join(row["folder"], row["lfp"])
        assert os.path.exists(lfp_fpath)
    else:
        lfp_fpath = None
    
    has_nd2 = " " if nd2_fpath is None else "x"
    has_lv = " " if labview_fpath is None else "x"
    has_nikmeta = " " if nikmeta_fpath is None else "x"
    has_lfp = " " if lfp_fpath is None else "x"
    print(f"{uuid}\nnd2: [{has_nd2}] lv: [{has_lv}] nik ts: [{has_nikmeta}] lfp: [{has_lfp}]")
    
    experiment_type = row["experiment_type"]
    
    
    # process labview data (if present)
    ses = tps.TwoPhotonSession.init_and_process(nd2_path=nd2_fpath, nd2_timestamps_path=nikmeta_fpath, labview_path=labview_fpath, labview_timestamps_path = labview_tstamps_fpath, lfp_path = lfp_fpath, matlab_2p_folder=matlab_2p_folder)
    
    # extract mean fluorescence signal
    if nd2_fpath is not None:
        mean_fluo = ses.return_nikon_mean()
    
    
    delete_file = True
    try:
        with h5py.File(export_h5fpath, 'w') as hf:
            hf.attrs["uuid"] = uuid
            if nd2_fpath is not None:
                hf.create_dataset("mean_fluo", data=mean_fluo)
                if labview_fpath is not None:  # matching of Nikon and LabView can only happen if nd2 recording AND labview recording exist.
                    # Do not write NaN or pupil/reward/airpuff-related data, as we are not using them.
                    hf.create_dataset("round", data=ses.belt_dict["round"])
                    hf.create_dataset("speed", data=ses.belt_dict["speed"])            
                    hf.create_dataset("distance", data=ses.belt_dict["distance"])            
                    hf.create_dataset("distancePR", data=ses.belt_dict["distancePR"])            
                    hf.create_dataset("reflect", data=ses.belt_dict["reflect"])            
                    hf.create_dataset("licking", data=ses.belt_dict["licking"])            
                    hf.create_dataset("stripes", data=ses.belt_dict["stripes"])   
                    hf.create_dataset("stripesPR", data=ses.belt_dict["stripesPR"])            
                    hf.create_dataset("time", data=ses.belt_dict["time"])            
                    hf.create_dataset("timePR", data=ses.belt_dict["timePR"])            
                    hf.create_dataset("tsscn", data=ses.belt_dict["tsscn"])  
                    hf.create_dataset("running", data=ses.belt_dict["running"])            
                    hf.create_dataset("runtime", data=ses.belt_dict["runtime"])  # Warning: empty dataset if the mouse did not run

                    hf.create_dataset("running_scn", data=ses.belt_scn_dict["running"])
                    hf.create_dataset("round_scn", data=ses.belt_scn_dict["rounds"])
                    hf.create_dataset("speed_scn", data=ses.belt_scn_dict["speed"])
                    hf.create_dataset("totdistscn", data=ses.belt_scn_dict["totdist"])
                    hf.create_dataset("distance_scn", data=ses.belt_scn_dict["distance"])
                
        delete_file = False
        print(f"Saved file {export_h5fpath}.")
    finally:  # TODO: improve this handling of errors by looking up what kind of errors happen in try block
        if delete_file and os.path.exists(export_h5fpath):
            print(f"Attempting to delete incomplete file {export_h5fpath}...")
            os.remove(export_h5fpath)
            print(f"Success.")
    print()

Results file for 171693d0988c458a96c8198c7b8cfc28 already exists; skipping session.
Results file for a6099849121f44ccbec237037971ab57 already exists; skipping session.
Results file for e40f26d410ab452e8f8d59e5394ae0fe already exists; skipping session.
Results file for ae564f8c867f4f35aa971b6562c33a7c already exists; skipping session.
Results file for 65bff16a4cf04930a5cb14f489a8f99b already exists; skipping session.
Results file for 30dc55d1a5dc4b0286d132e72f208ca6 already exists; skipping session.
Results file for d7a5ac8e2bc74382b3db503a6a5a07a5 already exists; skipping session.
Results file for 5ea6fd9c4cb542dbbc1f65305725cede already exists; skipping session.
Results file for 06ebcf354f5c41519669f187e16de364 already exists; skipping session.
Results file for 73a27053f4bf4ae1b4ad96064b6dabc0 already exists; skipping session.
Results file for 2aa75aa234a749668eb896e7e00aa87a already exists; skipping session.
Results file for 79fb974821f34e3abdcf5ca650e1c0f4 already exists; skipping s

In [ ]:
for iterrow in df_grouping.iterrows():  # tuple of (index, row)
    i_row, row = iterrow
    uuid = row["uuid"]
    export_h5fpath = os.path.join(export_results_folder, str(uuid) + "_tps.h5")
    
    if os.path.exists(export_h5fpath):
        print(f"Results file for {uuid} already exists; skipping session.")
        continue
    if type(row["nd2"]) is not float:  # NaN entries are of type float
        nd2_fpath = os.path.join(row["folder"], row["nd2"])
        assert os.path.exists(nd2_fpath)
    else:
        nd2_fpath = None
    if type(row["labview"]) is not float:  # NaN entries are of type float
        labview_fpath = os.path.join(row["folder"], row["labview"])
        labview_tstamps_fpath = os.path.join(row["folder"], os.path.splitext(row["labview"])[0] + "time.txt")  # infer labview timestamps file name
        assert os.path.exists(labview_fpath)
    else:
        labview_fpath = None
        labview_tstamps_fpath = None
    if type(row["nikon_meta"]) is not float:  # NaN entries are of type float
        nikmeta_fpath = os.path.join(row["folder"], row["nikon_meta"])
        assert os.path.exists(nikmeta_fpath)
    else:
        nikmeta_fpath = None
    if type(row["lfp"]) is not float:  # NaN entries are of type float
        lfp_fpath = os.path.join(row["folder"], row["lfp"])
        assert os.path.exists(lfp_fpath)
    else:
        lfp_fpath = None
    
    has_nd2 = " " if nd2_fpath is None else "x"
    has_lv = " " if labview_fpath is None else "x"
    has_nikmeta = " " if nikmeta_fpath is None else "x"
    has_lfp = " " if lfp_fpath is None else "x"
    print(f"{uuid}\nnd2: [{has_nd2}] lv: [{has_lv}] nik ts: [{has_nikmeta}] lfp: [{has_lfp}]")
    
    experiment_type = row["experiment_type"]
    
    
    # process labview data (if present)
    ses = tps.TwoPhotonSession.init_and_process(nd2_path=nd2_fpath, nd2_timestamps_path=nikmeta_fpath, labview_path=labview_fpath, labview_timestamps_path = labview_tstamps_fpath, lfp_path = lfp_fpath, matlab_2p_folder=matlab_2p_folder, uuid=uuid)
    
    delete_file = True
    try:
        ses.export_hdf5(fpath=export_h5fpath)
        delete_file = False
        print(f"Saved file {export_h5fpath}.")
    finally:  # TODO: improve this handling of errors by looking up what kind of errors happen when saving to hdf5
        if delete_file and os.path.exists(export_h5fpath):
            print(f"Attempting to delete incomplete file {export_h5fpath}...")
            os.remove(export_h5fpath)
            print(f"Success.")
    print()

In [ ]:
example_row = df_grouping.iloc[8]

In [ ]:
example_row

In [ ]:
ex_nd2_fpath = os.path.join(example_row["folder"], example_row["nd2"])
ex_labview_fpath = os.path.join(example_row["folder"], example_row["labview"])
ex_nikmeta_fpath = os.path.join(example_row["folder"], example_row["nikon_meta"])
ex_labview_tstamps_fpath = os.path.join(example_row["folder"], os.path.splitext(example_row["labview"])[0] + "time.txt")
if type(example_row["lfp"]) is not float:
    ex_lfp_fpath = os.path.join(example_row["folder"], example_row["lfp"])
else:
    ex_lfp_fpath = None
ex_experiment_type = example_row["experiment_type"]
ex_uuid = example_row["uuid"]

In [ ]:
ex_ses = tps.TwoPhotonSession.init_and_process(nd2_path=ex_nd2_fpath, nd2_timestamps_path=ex_nikmeta_fpath, labview_path=ex_labview_fpath, labview_timestamps_path = ex_labview_tstamps_fpath, lfp_path = ex_lfp_fpath, matlab_2p_folder=matlab_2p_folder)

In [ ]:
ex_ses.belt_dict

In [ ]:
ex_ses.belt_scn_dict

In [ ]:
ex_ses.belt_dict["runtime"]